# Using Deep Learning to Identify Images of Fruits

I will be using deep learning techniques via the Keras API to train a Convolutional Neural Network (CNN) model on a dataset of fruit images, and evaluate this model's accuracy on a separate test dataset of fruit images.

I will be focusing on optimizing my CNN model to give me the highest possible accuracy on both my training & test data.

## Importing Training & Test Images

In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd
import skimage
from skimage import io, transform
from IPython.display import Image, display

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import LSTM, Input
from keras.models import Model
from keras.optimizers import Adam

img_size = 100
train_dir = './data/fruits/test/'
test_dir =  './data/fruits/train/'

def get_data(folder_path):
    imgs = []
    indices = []
    labels = []
    for idx, folder_name in enumerate(os.listdir(folder_path)[:10]):
        if not folder_name.startswith('.'):
            labels.append(folder_name)
            for file_name in tqdm(os.listdir(folder_path + folder_name)):
                if not file_name.startswith('.'):
                    img_file = io.imread(folder_path + folder_name + '/' + file_name)
                    if img_file is not None:
                        img_file = transform.resize(img_file, (img_size, img_size))
                        imgs.append(np.asarray(img_file))
                        indices.append(idx)
    imgs = np.asarray(imgs)
    indices = np.asarray(indices)
    labels = np.asarray(labels)
    return imgs, indices, labels

X_train, y_train, train_labels = get_data(train_dir)
X_test, y_test, test_labels = get_data(test_dir)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
  0%|          | 0/166 [00:00<?, ?it/s]/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
100%|██████████| 490/490 [00:01<00:00, 359.94it/s]


## Data Wrangling

In [2]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train:', y_train)
print('y_test:', y_test)
# print('First image - X_train:', X_train[0])

num_categories = len(np.unique(y_train))

new_X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]).astype('float32')
new_X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3]).astype('float32')
new_y_train = keras.utils.to_categorical(y_train, num_categories)
new_y_test = keras.utils.to_categorical(y_test, num_categories)

X_train shape: (1709, 100, 100, 3)
X_test shape: (5093, 100, 100, 3)
y_train: [0 0 0 ... 9 9 9]
y_test: [0 0 0 ... 9 9 9]


## Exploratory Data Analysis

In [ ]:
def display_imgs(folder_path):
    for idx, folder_name in enumerate(os.listdir(folder_path)):
        if idx % 25 == 0:
            if not folder_name.startswith('.'):
                for idx2, file_name in enumerate(tqdm(os.listdir(folder_path + folder_name))):
                    if idx2 % 75 == 0:
                        if not file_name.startswith('.'):
                            img_filename = folder_path + folder_name + '/' + file_name
                            display(Image(filename=img_filename))

### Examples of Training Images

In [ ]:
display_imgs(train_dir)

### Examples of Test Images

In [ ]:
display_imgs(test_dir)

## Initial Model Selection
### Convolutional Neural Network (CNN)

In [3]:
def evaluate_model(model, batch_size, epochs):
    history = model.fit(new_X_train, new_y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(new_X_test, new_y_test))
    score = model.evaluate(new_X_test, new_y_test, verbose=0)
    print('***Metrics Names***', model.metrics_names)
    print('***Metrics Values***', score)

In [ ]:
convolutional = Sequential()

convolutional.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional.add(Conv2D(64, (3, 3), activation='relu'))
convolutional.add(MaxPooling2D(pool_size=(2, 2)))
convolutional.add(Dropout(0.25))

convolutional.add(Flatten())
convolutional.add(Dense(128, activation='relu'))
convolutional.add(Dropout(0.5))
convolutional.add(Dense(num_categories, activation='softmax'))

convolutional.summary()
convolutional.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional, 128, 5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               18874496  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

From the above epochs, I can already see that my model is over-fitting on my training data from the 2nd epoch on – we can see that the model's testing accuracy is a full 13% higher than its validation accuracy (~93% and ~80%, respectively). The highest validation accuracy comes at the 3rd epoch (~83%), and goes slightly lower in subsequent epochs. 

Overall, my model is reliably performing at ~81% accuracy, give or take ~2% percent, depending on the model run and epoch. I will now be attempting to optimize this model. 

## Optimizing the CNN Model
### Strategy 0 – Increase Dropout Rates to Counter Overfitting

In [ ]:
convolutional = Sequential()

convolutional.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional.add(Conv2D(64, (3, 3), activation='relu'))
convolutional.add(MaxPooling2D(pool_size=(2, 2)))
convolutional.add(Dropout(0.6))

convolutional.add(Flatten())
convolutional.add(Dense(128, activation='relu'))
convolutional.add(Dropout(0.6))
convolutional.add(Dense(num_categories, activation='softmax'))

convolutional.summary()
convolutional.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional, 128, 7)

Increasing the dropout does help with overfitting – it is still overfitting in the 2nd epoch by ~12%, but that overfit percentage goes down to 5% in the 3rd epoch. Overall, this model seems to be hovering at ~84% validation accuracy.

### Strategy 1 – Use Different Loss Functions [Not Successful]

In [ ]:
def run_with_loss(loss):
    convolutional = Sequential()

    convolutional.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
    convolutional.add(Conv2D(64, (3, 3), activation='relu'))
    convolutional.add(MaxPooling2D(pool_size=(2, 2)))
    convolutional.add(Dropout(0.25))

    convolutional.add(Flatten())
    convolutional.add(Dense(128, activation='relu'))
    convolutional.add(Dropout(0.5))
    convolutional.add(Dense(num_categories, activation='softmax'))

    convolutional.summary()
    convolutional.compile(loss=loss, optimizer=Adam(), metrics=['accuracy'])

    evaluate_model(convolutional, 128, 5)

losses = ['mean_squared_error', 'mean_absolute_error', 'mean_squared_logarithmic_error']

for loss in losses:
    run_with_loss(loss)

None of these other loss functions show a significant improvement over the original loss function of 'categorical_crossentropy'. Of the 3 attempted, 'mean_squared_error' performed the best, coming in with a validation accuracy of ~73% before overfitting.

### Strategy 2 – Add More Convolutional Layers [Successful]

In [ ]:
convolutional_2 = Sequential()

convolutional_2.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_2.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_2.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
convolutional_2.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_2.add(Dropout(0.6))

# CHANGE
# convolutional_2.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
# convolutional_2.add(Conv2D(256, (3, 3), activation='relu'))
# convolutional_2.add(MaxPooling2D(pool_size=(2, 2)))
# convolutional_2.add(Dropout(0.25))

convolutional_2.add(Flatten())
convolutional_2.add(Dense(256, activation='relu'))
convolutional_2.add(Dropout(0.6))
convolutional_2.add(Dense(num_categories, activation='softmax'))

convolutional_2.summary()
convolutional_2.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional_2, 128, 5)

### Strategy 3 – Flatten Before Dropping Out [Not Successful]

In [ ]:
convolutional_3 = Sequential()

convolutional_3.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_3.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_3.add(MaxPooling2D(pool_size=(2, 2)))

# CHANGE
convolutional_3.add(Flatten())
convolutional_3.add(Dropout(0.25))

convolutional_3.add(Dense(128, activation='relu'))
convolutional_3.add(Dropout(0.5))
convolutional_3.add(Dense(num_categories, activation='softmax'))

convolutional_3.summary()
convolutional_3.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional_3, 128, 5)

### Strategy 4 – Change Batch Size (Increase & Decrease)  [Not Successful, Not Successful]

In [ ]:
convolutional_4 = Sequential()

convolutional_4.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_4.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_4.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_4.add(Dropout(0.25))

convolutional_4.add(Flatten())
convolutional_4.add(Dense(128, activation='relu'))
convolutional_4.add(Dropout(0.5))
convolutional_4.add(Dense(num_categories, activation='softmax'))

convolutional_4.summary()
convolutional_4.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

# CHANGE
evaluate_model(convolutional_4, 512, 5)

In [ ]:
convolutional_4_v2 = Sequential()

convolutional_4_v2.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_4_v2.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_4_v2.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_4_v2.add(Dropout(0.25))

convolutional_4_v2.add(Flatten())
convolutional_4_v2.add(Dense(128, activation='relu'))
convolutional_4_v2.add(Dropout(0.5))
convolutional_4_v2.add(Dense(num_categories, activation='softmax'))

convolutional_4_v2.summary()
convolutional_4_v2.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

# CHANGE
evaluate_model(convolutional_4_v2, 32, 5)

### Strategy 5 – Increase & Decrease Kernel Size [Not Successful, Successful]

In [ ]:
convolutional_5 = Sequential()

# CHANGE
convolutional_5.add(Conv2D(32, kernel_size=(4,4), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))

convolutional_5.add(Conv2D(64, (4, 4), activation='relu'))
convolutional_5.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_5.add(Dropout(0.25))

convolutional_5.add(Flatten())
convolutional_5.add(Dense(128, activation='relu'))
convolutional_5.add(Dropout(0.5))
convolutional_5.add(Dense(num_categories, activation='softmax'))

convolutional_5.summary()
convolutional_5.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional_5, 128, 5)

In [ ]:
convolutional_5_v2 = Sequential()

# CHANGE
convolutional_5_v2.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))

convolutional_5_v2.add(Conv2D(64, (2, 2), activation='relu'))
convolutional_5_v2.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_5_v2.add(Dropout(0.25))

convolutional_5_v2.add(Flatten())
convolutional_5_v2.add(Dense(128, activation='relu'))
convolutional_5_v2.add(Dropout(0.5))
convolutional_5_v2.add(Dense(num_categories, activation='softmax'))

convolutional_5_v2.summary()
convolutional_5_v2.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional_5_v2, 128, 5)

### Strategy 6 – Add Max Pooling Layer Between Convolutional Layers [Not Successful]

In [ ]:
convolutional_6 = Sequential()

convolutional_6.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_6.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_6.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_6.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_6.add(Dropout(0.25))

convolutional_6.add(Flatten())
convolutional_6.add(Dense(128, activation='relu'))
convolutional_6.add(Dropout(0.5))
convolutional_6.add(Dense(num_categories, activation='softmax'))

convolutional_6.summary()
convolutional_6.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional_6, 128, 5)

### Strategy 7 – Adjust Learning Rate of Optimizer [Successful]

In [ ]:
convolutional_7 = Sequential()

convolutional_7.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_7.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_7.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_7.add(Dropout(0.25))

convolutional_7.add(Flatten())
convolutional_7.add(Dense(128, activation='relu'))
convolutional_7.add(Dropout(0.5))
convolutional_7.add(Dense(num_categories, activation='softmax'))

convolutional_7.summary()
convolutional_7.compile(loss="categorical_crossentropy", optimizer=Adam(lr=.0001), metrics=['accuracy'])

evaluate_model(convolutional_7, 128, 5)

### Strategy 8 – Increase Dropout Rates [Successful]

In [ ]:
convolutional_8_v2 = Sequential()

convolutional_8_v2.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_8_v2.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_8_v2.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_8_v2.add(Dropout(0.5))

convolutional_8_v2.add(Flatten())
convolutional_8_v2.add(Dense(128, activation='relu'))
convolutional_8_v2.add(Dropout(0.6))
convolutional_8_v2.add(Dense(num_categories, activation='softmax'))

convolutional_8_v2.summary()
convolutional_8_v2.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

evaluate_model(convolutional_8_v2, 128, 5)

## Finalizing Model

Add all strategies that worked (additional convolutional layers, smaller learning rate, smaller kernel size), alongside Batch Normalization, for a final optimized model.

In [ ]:
convolutional_final = Sequential()

convolutional_final.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3],)))
convolutional_final.add(Conv2D(64, (3, 3), activation='relu'))
convolutional_final.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_final.add(Dropout(0.4))

# CHANGE
convolutional_final.add(Conv2D(128, (3, 3), activation='relu'))
convolutional_final.add(Conv2D(256, (3, 3), activation='relu'))
convolutional_final.add(MaxPooling2D(pool_size=(2, 2)))
convolutional_final.add(Dropout(0.4))

convolutional_final.add(Flatten())
convolutional_final.add(Dense(512, activation='relu'))
convolutional_final.add(Dropout(0.6))
convolutional_final.add(BatchNormalization())
convolutional_final.add(Dense(num_categories, activation='softmax'))

convolutional_final.summary()

# CHANGE
convolutional_final.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.005), metrics=['accuracy'])

# CHANGE
evaluate_model(convolutional_final, 128, 5)

* 		drop out layers (add and remove; adjust percentage)
* 		play with convolutional layer numbers
* 		100 epochs for 10,000 images

tensor board – good for visualizing model performance (good to add to deep learning portfolio)
do visualizations that are similar to what is shown in tensorboard

### Evaluating Finalized Model


In [ ]:
y_pred = convolutional_final.predict(new_X_test, batch_size=None, verbose=0, steps=None).argmax(axis=-1)
res_crosstab = pd.crosstab(y_pred, y_test)

dict_idx_fruit = {idx: label for idx, label in enumerate(test_labels)}
print(dict_idx_fruit)

res_crosstab

In [ ]:
for idx in range(num_categories):
    accuracy = res_crosstab.loc[idx, idx] / res_crosstab.loc[:, idx].sum()
    flag = '***LOW***' if accuracy < 0.8 else ''
    print(dict_idx_fruit[idx])
    print('   ', flag, 'accuracy –', round(accuracy * 100, 2), '%')